First lets do the avg-based FFT, based on these basis functions:


$\bar{s}^k_i = \left< sin(2 \pi k x) \right>_i$

$\bar{c}^k_i = \left< cos(2 \pi k x) \right>_i$

where $x_i = i / N$ for $i \in [0, \dots, N-1]$, and  $\left< \cdot\right>_i$ means average over $[x_i,x_{i+1}]$

In [1]:
import numpy as np
import numpy.fft as fft
import numpy.linalg as LA

# NOTE - these lines will shut off wrapping so scrollbar instead
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth=1000)

In [2]:
# Number of points in domain - don't forget x=1 is periodic wrap of x=0
N=8

In [3]:
# Create the node-based functions
x_node = np.linspace(0,1-1/N,N)  # x node values from 0 ... not including 1
dx = 1/N  # cell spacing
basis_k = np.zeros((N,N))  # basis functions
basis_k[:,0] = np.ones(N)  # k=0 mode
key_k = ['c0']  # labels for modes
k_vals = np.zeros(N)  # for k numbers
# Fill in sin/cos modes for k between 0 and N/2
x_lo = x_node[0:N]  # left cell edges
x_hi = np.append(x_node[1:N],[1])  # right cell edges

In [4]:
# Initialize the avg of each basis
max_k = int(N/2)
for k in range(1,max_k):
    k_vals[2*k-1] = k
    k_vals[2*k] = k
    key_k.append(f's{k}')
    key_k.append(f'c{k}')
    tpik = 2*np.pi*k
    basis_k[:,2*k-1] = -1/(tpik*dx)*(np.cos(tpik*x_hi) - np.cos(tpik*x_lo))
    basis_k[:,2*k] = 1/(tpik*dx)*(np.sin(tpik*x_hi) - np.sin(tpik*x_lo))

# cell avg of +/- highest sin mode
tpik = 2*np.pi*max_k
basis_k[:,N-1] = -1/(tpik*dx)*(np.cos(tpik*x_hi) - np.cos(tpik*x_lo))
key_k.append(f's{max_k}') 
k_vals[N-1]=max_k
basis_k  # each row is a mode for coef 1 in front of that sin/cos wave number k

array([[ 1.        ,  0.37292323,  0.90031632,  0.63661977,  0.63661977,  0.72451862,  0.30010544,  0.63661977],
       [ 1.        ,  0.90031632,  0.37292323,  0.63661977, -0.63661977, -0.30010544, -0.72451862, -0.63661977],
       [ 1.        ,  0.90031632, -0.37292323, -0.63661977, -0.63661977, -0.30010544,  0.72451862,  0.63661977],
       [ 1.        ,  0.37292323, -0.90031632, -0.63661977,  0.63661977,  0.72451862, -0.30010544, -0.63661977],
       [ 1.        , -0.37292323, -0.90031632,  0.63661977,  0.63661977, -0.72451862, -0.30010544,  0.63661977],
       [ 1.        , -0.90031632, -0.37292323,  0.63661977, -0.63661977,  0.30010544,  0.72451862, -0.63661977],
       [ 1.        , -0.90031632,  0.37292323, -0.63661977, -0.63661977,  0.30010544, -0.72451862,  0.63661977],
       [ 1.        , -0.37292323,  0.90031632, -0.63661977,  0.63661977, -0.72451862,  0.30010544, -0.63661977]])

In [5]:
print(key_k)
print(k_vals)

['c0', 's1', 'c1', 's2', 'c2', 's3', 'c3', 's4']
[0. 1. 1. 2. 2. 3. 3. 4.]


In [6]:
# Create a Laplacian
ones = np.full(N,1)
lapl = np.diagflat(-2*ones,0) + np.diagflat(ones[1:N],1) \
       + np.diagflat(ones[1:N],-1)
lapl[0,N-1] = 1
lapl[N-1,0] = 1
print('Laplacian = \n' + str(lapl))

Laplacian = 
[[-2  1  0  0  0  0  0  1]
 [ 1 -2  1  0  0  0  0  0]
 [ 0  1 -2  1  0  0  0  0]
 [ 0  0  1 -2  1  0  0  0]
 [ 0  0  0  1 -2  1  0  0]
 [ 0  0  0  0  1 -2  1  0]
 [ 0  0  0  0  0  1 -2  1]
 [ 1  0  0  0  0  0  1 -2]]


In [7]:
# See what L * our modes are, and the eigenvalues
Lv_ratio = np.zeros((N,N))
eigvals = np.zeros(N)
for k in range(0,N):
    Lv = lapl@basis_k[:,k]
    Lv_ratio[k,:] = Lv / basis_k[:,k]
    # This will make sure all the entry-wise values are the same
    if np.isclose(Lv_ratio[k,:], Lv_ratio[k,0], 1e-14).all():
        eigvals[k] = Lv_ratio[k,0]

print(eigvals)
test_eigvals = -2 + 2*np.cos(2*np.pi*k_vals/N)
# Check theoretical eigenvalues vs. what we calculated from cell-averages
assert np.isclose(eigvals, test_eigvals, 1e-14).all()

print('\nLaplacian eigenvector/value test passed!')

[ 0.         -0.58578644 -0.58578644 -2.         -2.         -3.41421356 -3.41421356 -4.        ]

Laplacian eigenvector/value test passed!


In [8]:
# Show they're orthogonal, too
orth_k = np.round(basis_k.T@basis_k,14)
print(orth_k)

# NOTE: we need these for coef scaling on the transform!
scale_k = np.diag(orth_k)
scale_k

[[ 8.          0.         -0.         -0.         -0.          0.         -0.          0.        ]
 [ 0.          3.79856481 -0.          0.          0.          0.          0.         -0.        ]
 [-0.         -0.          3.79856481 -0.         -0.          0.         -0.         -0.        ]
 [-0.          0.         -0.          3.24227788 -0.          0.         -0.          0.        ]
 [-0.          0.         -0.         -0.          3.24227788  0.         -0.         -0.        ]
 [ 0.          0.          0.          0.          0.          2.45996202 -0.          0.        ]
 [-0.          0.         -0.         -0.         -0.         -0.          2.45996202 -0.        ]
 [ 0.         -0.         -0.          0.         -0.          0.         -0.          3.24227788]]


array([8.        , 3.79856481, 3.79856481, 3.24227788, 3.24227788, 2.45996202, 2.45996202, 3.24227788])

In [9]:
# For example, let's test for some combination of modes
y = 1*basis_k[:,0] - 2*basis_k[:,1] + .5*basis_k[:,N-1]
# Another way to create this is with matrix mult
test_coef = np.zeros(N)
test_coef[0] = 1
test_coef[1] = -2
test_coef[N-1] = .5
test_y = test_coef@basis_k.T
mleh = basis_k@test_coef
assert np.isclose(mleh, test_y, 1e-14).all()

# This is the official representation of our fv_fft of sin/cos modes as a matrix
fv_fft_sc = np.diagflat(1/scale_k)@basis_k.T

print(np.round(fv_fft_sc@fv_fft_sc.T, 14)) # I put this here.
print(np.round(np.diagflat(1/scale_k), 14)) # I put this here.
print('sin/cos mode FV_FFT = ')
print(fv_fft_sc)
# coef = (basis_k.T@y) / scale_k
coef = fv_fft_sc@y
print('y =', y) # I put this here.
print(np.round(coef, 14))
print(np.round(LA.inv(basis_k) - fv_fft_sc, 14))
assert np.isclose(coef, test_coef, 1e-14).all()

print('\nTests passed')

[[ 0.125       0.         -0.         -0.         -0.          0.         -0.          0.        ]
 [ 0.          0.26325732 -0.          0.          0.          0.          0.          0.        ]
 [-0.         -0.          0.26325732 -0.         -0.         -0.         -0.         -0.        ]
 [-0.          0.         -0.          0.30842514  0.          0.         -0.          0.        ]
 [-0.          0.         -0.          0.          0.30842514  0.         -0.         -0.        ]
 [ 0.          0.         -0.          0.          0.          0.40651034 -0.          0.        ]
 [-0.          0.         -0.         -0.         -0.         -0.          0.40651034 -0.        ]
 [ 0.          0.         -0.          0.         -0.          0.         -0.          0.30842514]]
[[0.125      0.         0.         0.         0.         0.         0.         0.        ]
 [0.         0.26325732 0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0

In [10]:
# Last step is to output what the FFT output would be:
# cos mode k has real coefs +/- 1/2 in +/- k slot
# sin mode k has imag coefs -/+ j/2 in +/- k slot
# put these in the same order as fftshift( ..., axis=1) would
fft_coef_scale = np.zeros((N,N), dtype=complex)
fft_coef_scale[0, max_k] = 1 # k=0 mode in max_k+1 column
for k in range(1,max_k):
    # sin mode
    fft_coef_scale[2*k-1,max_k-k] = .5*1j
    fft_coef_scale[2*k-1,max_k+k] = -.5*1j
    # cos mode
    fft_coef_scale[2*k,max_k-k] = .5
    fft_coef_scale[2*k,max_k+k] = .5
fft_coef_scale[N-1, 0] = 1 # k=max_k mode in first column
fft_coef_scale
blah = 1 * fft_coef_scale
print(fft_coef_scale)
print(blah[::2])

[[ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   1. +0.j   0. +0.j   0. +0.j   0. +0.j ]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.5j  0. +0.j  -0. -0.5j  0. +0.j   0. +0.j ]
 [ 0. +0.j   0. +0.j   0. +0.j   0.5+0.j   0. +0.j   0.5+0.j   0. +0.j   0. +0.j ]
 [ 0. +0.j   0. +0.j   0. +0.5j  0. +0.j   0. +0.j   0. +0.j  -0. -0.5j  0. +0.j ]
 [ 0. +0.j   0. +0.j   0.5+0.j   0. +0.j   0. +0.j   0. +0.j   0.5+0.j   0. +0.j ]
 [ 0. +0.j   0. +0.5j  0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j  -0. -0.5j]
 [ 0. +0.j   0.5+0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0.5+0.j ]
 [ 1. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j ]]
[[0. +0.j 0. +0.j 0. +0.j 0. +0.j 1. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0.5+0.j 0. +0.j 0.5+0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0.5+0.j]]


In [11]:
# Convert our test coefs to fft coefs
test_fft_y = fft_coef_scale.T@test_coef
test_fft_y

array([0.5+0.j, 0. +0.j, 0. +0.j, 0. -1.j, 1. +0.j, 0. +1.j, 0. +0.j, 0. +0.j])

In [12]:
# Now do the whole thing with FFTs for cell avg modes now
xhat = fft.fft(basis_k.T)  # FFT operates on rows, apparently
print(xhat)
# xhat = np.round(xhat,15)
xhat = np.round(fft.fftshift(xhat, axes=1),14)
xhat = xhat.T
print(xhat)  # as column vectors of mode coefs

[[ 8.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j  0.00000000e+00-0.00000000e+00j  0.00000000e+00-0.00000000e+00j  0.00000000e+00-0.00000000e+00j]
 [ 5.55111512e-17+0.00000000e+00j  1.49169291e+00-3.60126526e+00j  2.77555756e-16+0.00000000e+00j -4.44089210e-16+2.76130797e-16j  5.55111512e-17+0.00000000e+00j -4.44089210e-16-2.76130797e-16j  2.77555756e-16-0.00000000e+00j  1.49169291e+00+3.60126526e+00j]
 [-2.77555756e-16+0.00000000e+00j  3.60126526e+00+1.49169291e+00j  5.55111512e-17+0.00000000e+00j  1.70910491e-16+4.44089210e-16j -2.77555756e-16+0.00000000e+00j  1.70910491e-16-4.44089210e-16j  5.55111512e-17-0.00000000e+00j  3.60126526e+00-1.49169291e+00j]
 [ 0.00000000e+00+0.00000000e+00j -1.24491566e-16+1.43539982e-16j  2.54647909e+00-2.54647909e+00j  3.46536171e-16-3.00549228e-16j  4.44089210e-16+0.00000000e+00j  3.46536171e-16+3.00549228e-16j  2.54647909e+00+2.54647909e+0

In [13]:
# Now we need to figure out what the phase/amplitude error is:
# cos mode k has real coefs +/- 1/2 in +/- k slot
# sin mode k has imag coefs -/+ j/2 in +/- k slot
# This is the fft entry for the sin k=1 mode, which 
N*np.angle(1.49169291-3.60126526j)/(2*np.pi)  # note the N scaling

-1.5000000007227763

In [14]:
# This is the entry for the cos k=1 mode, which should be real
N*np.angle(3.60126526+1.49169291j)/(2*np.pi)

0.4999999992772236

In [15]:
# Let's just cheat and find the complex matrix that fixes applitude/phase
# Note, everything needs to be a column vector
print('xhat =\n', xhat)
print('fft_coef_scale.T =\n', fft_coef_scale.T)
print('np.linalg.inv(xhat) =\n', np.linalg.inv(xhat))
fft_fix = fft_coef_scale.T@np.linalg.inv(xhat)
fft_fix = np.round(fft_fix,16)
fix_xhat = np.round(fft_fix@xhat,15)
# Compare these to the original ones we wanted
print('fft_fix =\n', fft_fix)
print('fix_xhat =\n', fix_xhat)
assert(np.isclose(fft_coef_scale.T, fix_xhat, 1e-15).all())

xhat =
 [[ 0.        +0.j          0.        +0.j         -0.        +0.j          0.        +0.j          0.        +0.j          0.        +0.j         -0.        +0.j          5.09295818+0.j        ]
 [ 0.        -0.j         -0.        -0.j          0.        -0.j          0.        +0.j          0.        -0.j          2.89807448+1.20042175j  1.20042175-2.89807448j  0.        -0.j        ]
 [ 0.        -0.j          0.        -0.j          0.        -0.j          2.54647909+2.54647909j  2.54647909-2.54647909j  0.        -0.j         -0.        +0.j          0.        -0.j        ]
 [ 0.        -0.j          1.49169291+3.60126526j  3.60126526-1.49169291j -0.        -0.j         -0.        +0.j          0.        +0.j         -0.        +0.j          0.        -0.j        ]
 [ 8.        +0.j          0.        +0.j         -0.        +0.j          0.        +0.j         -0.        +0.j         -0.        +0.j         -0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j

In [16]:
# Note that fft_fix is diagonal, and the phase shift is just -pi*k/N
print(fft_fix)
fvfft_fix = np.diag(fft_fix)
# the phase shift is just -pi*k/N!
print(N*np.angle(fvfft_fix)/(np.pi))
fvfft_phase = -np.linspace(-max_k,max_k-1,N)
fvfft_phase[0]=0  # highest k mode has no phase error
print(fvfft_phase)
fvfft_phase = np.exp(np.pi*fvfft_phase*1j/N)
print("fvfft_phase correction (divide by)")
print(fvfft_phase)

[[0.19634954+0.j         0.        +0.j         0.        +0.j         0.        +0.j         0.        +0.j         0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.06099798+0.14726216j 0.        +0.j         0.        +0.j         0.        +0.j         0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.09817477+0.09817477j 0.        +0.j         0.        +0.j         0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j         0.11850743+0.04908739j 0.        +0.j         0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j         0.        +0.j         0.125     +0.j         0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j   

In [17]:
# This is just the amplitudes now, no phase errors - what's the formula for the amplitude scaling?
print("No phase - just amplitude:")
fvfft_amp = np.round(fvfft_fix / fvfft_phase,15)
print(fvfft_amp)
fvfft_amp  # there's probably an analytic expression for this, TBD

No phase - just amplitude:
[0.19634954+0.j 0.15939541+0.j 0.13884009-0.j 0.12827152+0.j 0.125     +0.j 0.12827152-0.j 0.13884009+0.j 0.15939541-0.j]


array([0.19634954+0.j, 0.15939541+0.j, 0.13884009-0.j, 0.12827152+0.j, 0.125     +0.j, 0.12827152-0.j, 0.13884009+0.j, 0.15939541-0.j])

In [18]:
print("These are the original 1*c0 + -2*s1 + .5*c4 values:")
print(test_y)
coef_fft = fft.fft(test_y)  # FFT operates on rows, apparently
coef_fft = np.round(fft.fftshift(coef_fft),14)
print("\nThese are the raw fft of them (note phase/amplitude error):")
print(coef_fft)

print("\nThese are the fvfft fix * the raw fft:")
coef_fft_avg = np.round(coef_fft * fvfft_fix,14)
print(coef_fft_avg)

print("\nThese are the original coefs using sin/cos mode transform matrix:")
print(test_coef)
assert(np.isclose(coef_fft_avg, fft_coef_scale.T@test_coef, 1e-14).all())

print("\nPasses fvfft transform vs. original inputs")

These are the original 1*c0 + -2*s1 + .5*c4 values:
[ 0.57246343 -1.11894252 -0.48232275 -0.06415634  2.06415634  2.48232275  3.11894252  1.42753657]

These are the raw fft of them (note phase/amplitude error):
[ 2.54647909+0.j          0.        +0.j         -0.        -0.j         -2.98338583-7.20253053j  8.        +0.j         -2.98338583+7.20253053j -0.        +0.j          0.        -0.j        ]

These are the fvfft fix * the raw fft:
[0.5+0.j 0. +0.j 0. -0.j 0. -1.j 1. +0.j 0. +1.j 0. +0.j 0. -0.j]

These are the original coefs using sin/cos mode transform matrix:
[ 1.  -2.   0.   0.   0.   0.   0.   0.5]

Passes fvfft transform vs. original inputs
